In [19]:
import pandas as pd
import numpy as np
import re
import ast
import string
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel


In [20]:
df = pd.read_parquet('data/movies.pq')
df.sample(5)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45429 entries, 0 to 45428
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4495 non-null   object 
 1   budget                 45429 non-null  float64
 2   genres                 45429 non-null  object 
 3   id                     45429 non-null  int64  
 4   original_language      45418 non-null  object 
 5   overview               45429 non-null  object 
 6   popularity             45429 non-null  object 
 7   production_companies   45429 non-null  object 
 8   production_countries   45429 non-null  object 
 9   release_date           45429 non-null  object 
 10  revenue                45429 non-null  float64
 11  runtime                45183 non-null  float64
 12  spoken_languages       45429 non-null  object 
 13  status                 45349 non-null  object 
 14  tagline                20416 non-null  object 
 15  ti

In [21]:

# df['production_countries'].info()
#df['United States of America' in df['production_countries'].cont]
#'United States of America' in df['production_countries'][0]

df.title = df.title.astype(str)
df = df [df['production_countries'].str.contains('United States of America')]
df = df [df['release_year']>1950]
print(df.shape)
df  = df.drop_duplicates(subset='title', keep='last')
df.shape



(18410, 21)


(17724, 21)

In [22]:
def clean_text(text):
    '''Make text lowercase,remove punctuation
    .'''
    text = str(text).lower().replace('collection','')
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    return text

df['belongs_to_collection'].fillna('', inplace=True)
df['content'] = df['title'] + " " + df['overview'] + " " + df['genres'] + ' ' +\
                df['belongs_to_collection'] + ' ' + df['director']


df['content'] = df['content'].apply(lambda x:clean_text(x))


In [23]:
df['content']

0        toy story led by woody andys toys live happily...
1        jumanji when siblings judy and peter discover ...
2        grumpier old men a family wedding reignites th...
3        waiting to exhale cheated on mistreated and st...
4        father of the bride part ii just when george b...
                               ...                        
45418    the morning after the morning after is a featu...
45422    the burkittsville 7 a film archivist revisits ...
45423    caged heat 3000 its the year 3000 ad the world...
45424    robin hood yet another version of the classic ...
45426    betrayal when one of her hits goes wrong a pro...
Name: content, Length: 17724, dtype: object

In [24]:
df["ids"]=[i for i in range(0,df.shape[0])]
df.tail(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,status,tagline,title,vote_average,vote_count,release_year,return,director,content,ids
45423,,0.0,['Science Fiction'],222848,en,It's the year 3000 AD. The world's most danger...,0.661558,[Concorde-New Horizons],['United States of America'],1995-01-01,...,Released,None,Caged Heat 3000,3.5,1.0,1995,0.0,Aaron Osborne,caged heat 3000 its the year 3000 ad the world...,17721
45424,,0.0,"['Drama', 'Action', 'Romance']",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[Westdeutscher Rundfunk (WDR), Working Title F...","['Canada', 'Germany', 'United Kingdom', 'Unite...",1991-05-13,...,Released,None,Robin Hood,5.7,26.0,1991,0.0,John Irvin,robin hood yet another version of the classic ...,17722
45426,,0.0,"['Action', 'Drama', 'Thriller']",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,[American World Pictures],['United States of America'],2003-08-01,...,Released,A deadly game of wits.,Betrayal,3.8,6.0,2003,0.0,Mark L. Lester,betrayal when one of her hits goes wrong a pro...,17723


In [25]:
#create the TF-IDF model
# MAX_DF     = 0.95
# MIN_DF     = 1#2
tfidf = TfidfVectorizer(token_pattern = r"\b\w{3,}\b", stop_words='english',  ngram_range=(1,2))

content = df.content#.dropna()

tfidf_matrix = tfidf.fit_transform(content)


cosine_similarities = None
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)


In [26]:
df.shape


(17724, 23)

In [27]:
# # Count Vectorizer

# content = q.dropna()
# count_vectorizer = CountVectorizer(token_pattern = r"\b\w{3,}\b", stop_words='english',  ngram_range=(1,2))
# count_matrix = count_vectorizer.fit_transform(content)
# cosine_similarities = None
# cosine_similarities = linear_kernel(count_matrix, count_matrix)



In [28]:
all_recomendations  = []

for i in df.ids:
    cosine_similarity_scores = list(enumerate(cosine_similarities[i]))
    cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)
    recomendations = []
    for s in cosine_similarity_scores[1:6]:
        title = df[df.ids==s[0]]['title'].to_string(index=False)
        recomendations.append(title)

    all_recomendations.append(recomendations)
     

    





In [29]:
df['recomendations'] = all_recomendations

In [30]:
df.to_parquet('data/movies_acotado.pq')

In [31]:
df.recomendations[0]

['Toy Story 2',
 'Toy Story 3',
 'Toy Story of Terror!',
 'Toy Story That Time Forgot',
 'Small Fry']

In [32]:
i = 0
cosine_similarity_scores = list(enumerate(cosine_similarities[i]))
cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)
recomendations = []
for s in cosine_similarity_scores[1:6]:
    recomendations.append(df[df.ids==s[0]]['title'].to_string(index=False))
    print (df[df.ids==s[0]]['title'].to_string(index=False))

print (recomendations)



Toy Story 2
Toy Story 3
Toy Story of Terror!
Toy Story That Time Forgot
Small Fry
['Toy Story 2', 'Toy Story 3', 'Toy Story of Terror!', 'Toy Story That Time Forgot', 'Small Fry']


In [33]:

cosine_similarity_scores = list(enumerate(cosine_similarities[i]))
cosine_similarity_scores = sorted(cosine_similarity_scores, key=lambda x: x[1], reverse=True)
cosine_similarity_scores 
df.loc[df.ids==0]['title'].to_string(index=False)


'Toy Story'

In [40]:
#del df
df = pd.read_parquet('data/movies_acotado.pq')
# df.sample(3)
# ast.literal_eval(df.recomendations[0])
peli = 'Toy Story'
peli = df[df.title == peli]
list(peli.iloc[0].recomendations)
# a  = print(peli.recomendations.to_string(index=False))
# print (a)

# ast.literal_eval(peli.recomendations.to_string(index=False))

#recomendations = list(peli.recomendations)

['Toy Story 2',
 'Toy Story 3',
 'Toy Story of Terror!',
 'Toy Story That Time Forgot',
 'Small Fry']